<a href="https://colab.research.google.com/github/RodrigoOBC/Notebooks_RedesNeurais/blob/main/flower_cnn_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base de dados encontrada no link 

### https://www.kaggle.com/alxmamaev/flowers-recognition

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
import random
import cv2
from tensorflow.keras import layers, models
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import pickle
import zipfile
tf.__version__

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 32
IMAGE_SIZE = [128, 128]
EPOCHS = 25

In [ ]:
image = cv2.imread('/kaggle/input/flowers-recognition/flowers/rose/7461878736_a55f749faf_n.jpg')

In [ ]:
print(image.shape)

In [ ]:
imgplot = plt.imshow(image)
plt.show()

In [ ]:
GCS_PATH = "/kaggle/input/flowers-recognition/flowers"

CLASS_NAMES = np.array([str(tf.strings.split(item, os.path.sep)[-1].numpy())[2:-1]
                        for item in tf.io.gfile.glob(str(GCS_PATH + "*/*"))])
CLASS_NAMES

In [ ]:
files_count = []
for i,f in enumerate(CLASS_NAMES):
    folder_path = os.path.join(GCS_PATH, f)
    for path in os.listdir(os.path.join(folder_path)):
        files_count.append(['{}/{}'.format(folder_path,path), f, i])
flowers_df = pd.DataFrame(files_count, columns=['filepath', 'class_name', 'label'])
flowers_df.head()

In [ ]:
flowers_df.class_name.value_counts()

In [ ]:
quantidade_por_class = 500
flowers_df = pd.concat([flowers_df[flowers_df['class_name']== i][:quantidade_por_class] for i in CLASS_NAMES])

In [ ]:
flowers_df.class_name.value_counts()


In [ ]:
image = cv2.imread(flowers_df.filepath[100])
imgplot = plt.imshow(image)
plt.show()

In [ ]:
X = flowers_df['filepath']
y = flowers_df['label']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


In [ ]:
#convert to tensor


x_train_tensor = tf.convert_to_tensor(x_train.values, dtype=tf.string)
y_train_tensor = tf.convert_to_tensor(y_train.values)

x_test_tensor = tf.convert_to_tensor(x_test.values, dtype=tf.string)
y_test_tensor = tf.convert_to_tensor(y_test.values)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((x_train_tensor, y_train_tensor))
test_data = tf.data.Dataset.from_tensor_slices((x_test_tensor, y_test_tensor))

In [ ]:
def map_fn(path, label):
    image = tf.image.decode_jpeg(tf.io.read_file(path))

    return image, label

#apply the function
train_data_img = train_data.map(map_fn)
test_data_img = test_data.map(map_fn)

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (15,5))
for i,l in train_data_img.take(1):
    ax[0].set_title('Imagem do dataset de treino');
    ax[0].imshow(i);
for i,l in test_data_img.take(1):
    ax[1].set_title('Imagem do dataset de test');
    ax[1].imshow(i);

In [ ]:
def preprocessing(image, label):
    """
    returns a image that is reshaped and normalized
    """
    image = tf.cast(image, tf.float32)
    image = image / 255.
    image = tf.image.resize(image, IMAGE_SIZE)
    
    return image, label


train_data_norm = train_data_img.map(preprocessing)
test_data_norm = test_data_img.map(preprocessing)

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (15,5))
for i,l in train_data_norm.take(1):
    ax[0].set_title('Imagem do dataset de treino');
    ax[0].imshow(i);
for i,l in test_data_norm.take(1):
    ax[1].set_title('Imagem do dataset de test');
    ax[1].imshow(i);

In [ ]:
train_batches = train_data_norm.batch(batch_size)
test_batches = test_data_norm.batch(batch_size)

for i, l in train_batches.take(1):
    print('Train Data Shape',i.shape)
for i, l in test_batches.take(1):
    print('Test Data Shape',i.shape)

In [ ]:
LeNet = models.Sequential()
LeNet.add(layers.Conv2D(6, (5,5), activation = 'relu', input_shape = (128, 128, 3)))
LeNet.add(layers.AveragePooling2D())
LeNet.add(layers.Conv2D(16, (5,5), activation = 'relu'))
LeNet.add(layers.AveragePooling2D())
LeNet.add(layers.Flatten())
LeNet.add(layers.Dense(255, activation='relu'))
LeNet.add(layers.Dropout(0.2))
LeNet.add(layers.Dense(124, activation='relu'))
LeNet.add(layers.Dropout(0.2))
LeNet.add(layers.Dense(84, activation='relu'))
LeNet.add(layers.Dense(43, activation='sigmoid'))
LeNet.summary()

In [ ]:
LeNet.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
history = LeNet.fit(train_batches, epochs=100,batch_size = 16,validation_data=(test_batches))

In [ ]:
def processar_imagem(path):
    img = tf.image.decode_image(tf.io.read_file(path))
    img = tf.cast(img, tf.float32)
    img = img / 255.
    img = tf.image.resize(img, IMAGE_SIZE)
    img_tensor = tf.convert_to_tensor(img)
    img_tensor = tf.expand_dims(img, axis=0)
    return img_tensor

In [ ]:
para_pred = processar_imagem(x_test.iloc[5])

prediction = LeNet.predict_classes(para_pred)[0]
print(("isso é uma " + str(CLASS_NAMES[prediction])).capitalize())
print(f'Na verdade ela é uma {CLASS_NAMES[y_test.iloc[5]]}')

In [ ]:
para_pred = processar_imagem(x_test.iloc[152])

prediction = LeNet.predict_classes(para_pred)[0]
print(("isso é uma " + str(CLASS_NAMES[prediction])).capitalize())
print(f'Na verdade ela é uma {CLASS_NAMES[y_test.iloc[152]]}')

In [ ]:
para_pred = processar_imagem(x_test.iloc[136])

prediction = LeNet.predict_classes(para_pred)[0]
print(("isso é uma " + str(CLASS_NAMES[prediction])).capitalize())
print(f'Na verdade ela é uma {CLASS_NAMES[y_test.iloc[136]]}')